In [1]:
import numpy as np
import random

# یال های موجود در گراف 
edges = {
    (0, 1): (62, 50),
    (0, 2): (44, 90),
    (0, 3): (67, 10),
    (1, 2): (33, 25),
    (1, 4): (52, 90),
    (2, 3): (32, 10),
    (2, 4): (52, 40),
    (3, 4): (54, 100),
}

# نود های موجود در گراف و تنظیم پارامتر ها 
nodes = [0, 1, 2, 3, 4]
population_size = 10
repeat = 10 
hyper_pop = []
my_hyper_pop = []


# تابعی برای محاسبه کردن فیتنس های یک مسیر
def calculate_costs(path):
    f1_cost = 0
    f2_cost = 0
    for i in range(len(path) - 1):
        edge = (path[i], path[i + 1])
        if edge in edges:
            f1, f2 = edges[edge]
        else:
            edge = (path[i + 1], path[i])
            if edge in edges:
                f1, f2 = edges[edge]
            else:
                return [1000, 1000]
        f1_cost += f1
        f2_cost += f2

    if path[0] != 0 or path[-1] != 4 or len(path) != len(set(path)):
        return [500, 500]
    return [f1_cost, f2_cost]

# تابع میوتیشن
def swap_mutation(individual, p_mutation=0.15):
    # Determine if mutation should occur based on p_mutation
    if random.random() > p_mutation:
        return individual

    # Select two random positions in the individual to swap
    pos1 = random.randint(0, len(individual) - 1)
    pos2 = random.randint(0, len(individual) - 1)

    # Ensure the positions are different
    while pos1 == pos2:
        pos2 = random.randint(0, len(individual) - 1)

    # Perform the swap mutation
    mutated_individual = individual[:]
    mutated_individual[pos1], mutated_individual[pos2] = mutated_individual[pos2], mutated_individual[pos1]

    return mutated_individual

# تابع کراس اور 
def single_point_crossover(parent1, parent2):
    # Determine the shorter and longer parent
    if len(parent1) > len(parent2):
        shorter, longer = parent2, parent1
    else:
        shorter, longer = parent1, parent2

    # Select a crossover point within the range of the shorter parent
    crossover_point = random.randint(1, len(shorter) - 1)

    # Create offspring by combining the parents at the crossover point
    offspring1 = shorter[:crossover_point] + longer[crossover_point:]
    offspring2 = longer[:crossover_point] + shorter[crossover_point:]
    
    offspring1 = swap_mutation(offspring1)
    offspring2 = swap_mutation(offspring2)


    return offspring1, offspring2

# تابعی برای محاسبه اینکه کدام اعضا دیگری را دامینیت میکنند
def calculate_domination(population):
    population_costs = [calculate_costs(path) for path in population]

    def dominates(cost1, cost2):
        return (cost1[0] <= cost2[0] and cost1[1] < cost2[1]) or (cost1[0] < cost2[0] and cost1[1] <= cost2[1])

    domination = []
    for i, cost1 in enumerate(population_costs):
        dominated_count = 0
        dominates_list = []
        for j, cost2 in enumerate(population_costs):
            if i != j:
                if dominates(cost1, cost2):
                    dominates_list.append(j)
                elif dominates(cost2, cost1):
                    dominated_count += 1
        domination.append([dominated_count, dominates_list])
    return domination

# تابعی که اعضای جمعیت را میگیرد و آن ها را براساس فرانت دسته بندی میکند
def calculate_sorted_fronts(population):
    domination = calculate_domination(population)
    sorted_fronts = []

    while any(d[0] == 0 for d in domination):
        current_front = [i for i, d in enumerate(domination) if d[0] == 0]
        sorted_fronts.append(current_front)
        for i in current_front:
            domination[i][0] = -1  # Mark as processed
            for dominated in domination[i][1]:
                domination[dominated][0] -= 1

    return sorted_fronts

# تابعی برای محاسبه کردن معیار فاصله 
def calculate_crowding_distance(population, front):
    front_costs = np.array([calculate_costs(population[i]) for i in front])
    num_individuals = len(front)
    num_objectives = front_costs.shape[1]

    # مقداردهی اولیه فاصله‌های تراکم به صفر
    crowding_distances = np.zeros(num_individuals)

    for m in range(num_objectives):
        # مرتب‌سازی بر اساس هر هدف
        sorted_indices = np.argsort(front_costs[:, m])
        sorted_costs = front_costs[sorted_indices, m]

        # اختصاص مقدار بی‌نهایت به نقاط مرزی
        crowding_distances[sorted_indices[0]] = float('inf')
        crowding_distances[sorted_indices[-1]] = float('inf')

        # نرمال‌سازی هزینه‌ها
        norm = sorted_costs[-1] - sorted_costs[0]
        if norm == 0:
            norm = 1  # جلوگیری از تقسیم بر صفر

        for i in range(1, num_individuals - 1):
            crowding_distances[sorted_indices[i]] += (sorted_costs[i + 1] - sorted_costs[i - 1]) / norm

    return crowding_distances

# تابعی که اعضای جمعیت را بر اساس فرانت دسته بندی میکند و اعضایی که در یک فرانت هستند را بر اساس معیار کرودینگ مرتب میکند 
def sort_population(population):
    fronts = calculate_sorted_fronts(population)
    crowding_fronts = []

    for front in fronts:
        crowding_distances = calculate_crowding_distance(population, front)
        crowding_front = [[population[ind], crowding_distances[i]] for i, ind in enumerate(front)]
        crowding_fronts.append(crowding_front)

    return crowding_fronts

# تابعی برای حذف کردن اعضای ضعیف تر بر اساس پرتو فرانت و کرودینگ دیستنس
def survivor_selection(population, number_of_survivors):
    # Get the fronts with crowding distances
    crowding_fronts = sort_population(population)
    
    # Initialize the list of survivors
    survivors = []
    
    for front in crowding_fronts:
        if len(survivors) + len(front) <= number_of_survivors:
            survivors.extend(front)
        else:
            # Sort the current front based on crowding distance in descending order
            front_sorted_by_crowding = sorted(front, key=lambda x: x[1], reverse=True)
            remaining_spots = number_of_survivors - len(survivors)
            survivors.extend(front_sorted_by_crowding[:remaining_spots])
            break
    
    # Extract the population members from the survivors list
    selected_population = [individual[0] for individual in survivors]
    
    return selected_population

# تابعی برای محاسبه مجموع 2 فیتنس یک عضو
def calculate_total_fitness(individual):
    # Assuming calculate_costs is defined and returns a list of two fitness values
    fitness_values = calculate_costs(individual)
    return sum(fitness_values)

# تابعی برای تورنمونت سلکشن از این تابع برای انتخاب والدین استفاده میشود 
def tournament_selection(population, tournament_size=2):
    selected_parents = []

    # Perform tournament selection
    for _ in range(2):  # Select two parents
        tournament = random.sample(population, tournament_size)
        best_individual = min(tournament, key=calculate_total_fitness)
        selected_parents.append(best_individual)

    return selected_parents

# تابعی برای مشخص کردن جمعیت اولیه
def initialize_population(population_size, nodes):
    population = []
    while len(population) < population_size:
        individual_length = random.randint(2, len(nodes))  # Length between 2 and the number of nodes
        individual = random.sample(nodes, individual_length)
        if individual[0] == 0 and individual[-1] == 4 and len(set(individual)) == len(individual):
            population.append(individual)
    return population

# تابعی برای تولید فرزندان جدید
def create_childrens(pop, number_of_child):
    number_of_child = number_of_child // 2
    for _ in range(number_of_child):
        parents = tournament_selection(pop)
        child1, child2 = single_point_crossover(parents[0], parents[1])
        pop.append(child1)
        if len(pop) < number_of_child + len(pop):
            pop.append(child2)

    return pop


In [2]:
# Run Algorithm
# تولید فرزندان + حذف اعضای ضعیف تر + تکرار مراحل قبلی
for j in range(repeat):
    population = initialize_population(population_size, nodes)
    for i in range(100):
        population = create_childrens(population, 6)
        population = survivor_selection(population , 10)
    hyper_pop.append(population)
    
    
for i in range(len(hyper_pop)):
    for j in hyper_pop[i]:
        my_hyper_pop.append(j)
    
    
complated_front = sort_population(my_hyper_pop)

unique_paths = list({tuple(path[0]) for path in complated_front[0]})
unique_paths = [list(path) for path in unique_paths]

In [5]:
#Result
print('pathes : ' , unique_paths)
print('costs :  ' , calculate_costs(unique_paths[0]) , calculate_costs(unique_paths[1]) , calculate_costs(unique_paths[2]))

pathes :  [[0, 3, 4], [0, 3, 2, 4], [0, 2, 4]]
costs :   [121, 110] [151, 60] [96, 130]
